In [39]:
import numpy as np
from termcolor import colored
from collections import defaultdict

### 1. Game implementation

In [2]:
np.random.seed(10)

reference for random seed: https://www.w3schools.com/python/ref_random_seed.asp

In [3]:
def make_grid(x, y, n):
    return np.random.randint(0, n, (x, y))

In [37]:
scorings = ['relative', 'difference']
# directions = [(x, y) for x in (-1, 0 ,1) for y in (-1, 0, 1)]
directions = [(0,1), (0,-1), (1,0), (-1,0)]

class Game:
    def __init__(self, x, y, n, scoring, seed=10):
        np.random.seed(seed)
        self.width = y
        self.height = x
        self.game_state = {'grid': make_grid(x,y,n), 'score': 0}
        self.is_end = False
        self.agent_state = {'pos': (0,0), 'history':set()}
        assert scoring in scorings
        self.scoring = scoring
        
    def step(self, velocity):
        if self.is_end:
            return
        # make sure direction is valid
        assert velocity in directions
        grid = self.game_state['grid']
        x, y = self.agent_state['pos']
        pos_t = (x + velocity[0], y + velocity[1])
        # make sure new position is inside grid
        if (pos_t[0] >= self.height or pos_t[0] < 0) or (pos_t[1] >= self.width or pos_t[1] < 0):
            return
        else:
            # update position
            self.agent_state['history'].add((x,y))
            self.agent_state['pos'] = pos_t
            self.game_state['seen'][pos_t] = 1
            # update score
            if self.scoring == 'relative':
                self.game_state['score'] += grid[x, y]
            elif self.scoring == 'difference':
                self.game_state['score'] += (abs(grid[x, y] - grid[pos_t[0], pos_t[1]]))
            
            # self.show_path()
            if (pos_t == (self.height, self.width)):
                self.is_end = true      
                
    def get_neighbours(self):
        grid = self.game_state['grid']
        x, y = self.agent_state['pos']
        neighbours = dict()
        for pos in directions:
            x, y = self.agent_state['pos']
            x += pos[0]
            y += pos[1]
            if not ((x >= self.height or x < 0) or (y >= self.width or y < 0)) and pos != (0,0):
                neighbours[(x,y)] = grid[x][y]
        return neighbours
        
    def show_grid(self):
        print('GAME GRID')
        grid = self.game_state['grid']
        for i in range(grid.shape[0]):
            print('| ', end='')
            for j in range(grid.shape[1]):
                print(colored(grid[i, j],'red'), end=' | ')
            print('\n')
    
    def show_path(self):
        score = self.game_state['score']
        print(f'GAME GRID WITH PATH\nSCORE: {score}')
        grid = self.game_state['grid']
        path = self.agent_state['history']
        for i in range(grid.shape[0]):
            print('| ', end='')
            for j in range(grid.shape[1]):
                if ((i, j) in path):
                    print(colored(grid[i, j], 'green'), end=' | ')
                elif (i, j) == self.agent_state['pos']:
                    print(''+colored(grid[i,j], 'red')+colored('*', 'blue'),end='| ')
                else: 
                    print(colored(grid[i, j], 'red'), end=' | ')
            print('\n')

In [5]:
game =  Game(3,3,10, 'relative')
game.show_grid()
game.show_path()

GAME GRID
| 9 | 4 | 0 | 

| 1 | 9 | 0 | 

| 1 | 8 | 9 | 

GAME GRID WITH PATH
SCORE: 0
| 9*| 4 | 0 | 

| 1 | 9 | 0 | 

| 1 | 8 | 9 | 



In [6]:
game.step((0,1))
game.show_path()

GAME GRID WITH PATH
SCORE: 9
| 9 | 4*| 0 | 

| 1 | 9 | 0 | 

| 1 | 8 | 9 | 



In [7]:
game.step((0,1))
game.show_path()

GAME GRID WITH PATH
SCORE: 13
| 9 | 4 | 0*| 

| 1 | 9 | 0 | 

| 1 | 8 | 9 | 



### 2. Heuristic algorithm

I will implement a greedy strategy that attempts maximise the short term reward of the agent by always moving first to cells that in the direction of the endpoint that have the shortest path values.

In [8]:
game =  Game(3,3,10, 'relative')
game.show_path()
game.get_neighbours()

GAME GRID WITH PATH
SCORE: 0
| 9*| 4 | 0 | 

| 1 | 9 | 0 | 

| 1 | 8 | 9 | 



{(0, 1): 4, (1, 0): 1}

In [30]:
game =  Game(3,3,10, 'difference')
n = 3
max_steps = 10
end = x-1, y-1
steps = 0
while (not game.is_end):
    steps += 1
    i, j = game.agent_state['pos']
    # first sort moves based on actual distance to end
    # 1. Find the n physically closest moves neighbours to the end
    grid = game.game_state['grid']
    visited = game.game_state['seen']
    neighbours = []
    # Our heuristics agent can only move down or to the right
    for pos in [(1,0),(0,1)]:
        x, y = game.agent_state['pos']
        x += pos[0]
        y += pos[1]
        if not ((x >= game.height or x < 0) or (y >= game.width or y < 0)) and pos != (0,0) and visited[x,y] == 0:
            neighbours.append((grid[x,y],pos[0],pos[1]))
    # 2. chose the move with the least score
    if len(neighbours) == 0:
        break
    best = sorted(neighbours, key=lambda x: x[0])[0]
    
    # 3. make the move
    game.step((best[1],best[2]))
    game.show_path()
    if (steps >= max_steps):
        break

GAME GRID WITH PATH
SCORE: 8
| 9 | 4 | 0 | 

| 1*| 9 | 0 | 

| 1 | 8 | 9 | 

GAME GRID WITH PATH
SCORE: 8
| 9 | 4 | 0 | 

| 1 | 9 | 0 | 

| 1*| 8 | 9 | 

GAME GRID WITH PATH
SCORE: 15
| 9 | 4 | 0 | 

| 1 | 9 | 0 | 

| 1 | 8*| 9 | 

GAME GRID WITH PATH
SCORE: 16
| 9 | 4 | 0 | 

| 1 | 9 | 0 | 

| 1 | 8 | 9*| 



### 3. Djikstras


For each cell in the grid, I need to work out every possible path to it, and the score of such path. I start with the first cell, then work out the next cells it can reach and recursively keep applying this until we have data for all cells.  Once this is complete, I start the end cell and select the next cell in the grid that has the shortest total path, then repeat until I have a path to the origin.

In [67]:
game =  Game(2,2,10, 'relative')
game.show_grid()

# Need to first find all possible paths
grid = game.game_state['grid']
paths = defaultdict(lambda: {'weight': 0, 'path':[]})
paths[(0,0)] = {'weight': 0, 'path':[]}
stack = [(0,0)]

# Apply depth first search
while len(stack) > 0:
    print(stack)
    pos0 = stack.pop()
    for pos1 in directions:
        x, y = pos0
        x += pos1[0]
        y += pos1[1]
        if not ((x >= game.height or x < 0) or (y >= game.width or y < 0)):
            weight = grid[x][y] + paths[pos0]['weight']
            path = paths[pos0]['path']
            # We do not want to cross the same node twice on the same path
            if not (x,y) in path:
                stack.append((x,y))
                # Compare with existing to find smallest path
                if (x,y) in paths:
                    if weight < paths[(x,y)]['weight']:
                        paths[(x,y)]['weight'] = weight
                        paths[(x,y)]['path'] = paths[pos0]['path']
                        paths[(x,y)]['path'].append(pos0)
                else:
                    paths[(x,y)] = {'weight':weight, 'path':paths[pos0]['path']+[pos0]}
                if ((x,y) == (game.height-1,game.width-1)):
                    break
                
            print((game.height-1,game.width-1))
           
# Backtrack from end to find optimal path


# Use path found as input to the game



GAME GRID
| 9 | 4 | 

| 0 | 1 | 

[(0, 0)]
(1, 1)
(1, 1)
[(0, 1), (1, 0)]
[(0, 1), (1, 1)]
(1, 1)
(1, 1)
[(0, 1), (0, 1)]
(1, 1)
[(0, 1), (1, 1)]
(1, 1)
(1, 1)
[(0, 1), (0, 1)]
(1, 1)
[(0, 1), (1, 1)]
(1, 1)
(1, 1)
[(0, 1), (0, 1)]
(1, 1)
[(0, 1), (1, 1)]
(1, 1)
(1, 1)
[(0, 1), (0, 1)]
(1, 1)
[(0, 1), (1, 1)]
(1, 1)
(1, 1)
[(0, 1), (0, 1)]
(1, 1)
[(0, 1), (1, 1)]
(1, 1)
(1, 1)
[(0, 1), (0, 1)]
(1, 1)
[(0, 1), (1, 1)]
(1, 1)
(1, 1)
[(0, 1), (0, 1)]
(1, 1)
[(0, 1), (1, 1)]
(1, 1)
(1, 1)
[(0, 1), (0, 1)]
(1, 1)
[(0, 1), (1, 1)]
(1, 1)
(1, 1)
[(0, 1), (0, 1)]
(1, 1)
[(0, 1), (1, 1)]
(1, 1)
(1, 1)
[(0, 1), (0, 1)]
(1, 1)
[(0, 1), (1, 1)]
(1, 1)
(1, 1)
[(0, 1), (0, 1)]
(1, 1)
[(0, 1), (1, 1)]
(1, 1)
(1, 1)
[(0, 1), (0, 1)]
(1, 1)
[(0, 1), (1, 1)]
(1, 1)
(1, 1)
[(0, 1), (0, 1)]
(1, 1)
[(0, 1), (1, 1)]
(1, 1)
(1, 1)
[(0, 1), (0, 1)]
(1, 1)
[(0, 1), (1, 1)]
(1, 1)
(1, 1)
[(0, 1), (0, 1)]
(1, 1)
[(0, 1), (1, 1)]
(1, 1)
(1, 1)
[(0, 1), (0, 1)]
(1, 1)
[(0, 1), (1, 1)]
(1, 1)
(1, 1)
[(0, 1), (0, 1)

KeyboardInterrupt: 

False